In [1]:
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
from numpy.linalg import solve, norm
from numpy.random import rand
from scipy.sparse import csr_matrix

In [2]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm

from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [17]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test_with_ids.csv")

In [4]:
len(X_test)

2345796

In [5]:
q_train = pd.DataFrame(np.concatenate((X_train.question1.unique(),X_train.question2.unique()), axis=0))

In [6]:
q_test = pd.DataFrame(np.concatenate((X_test.question1.unique(),X_test.question2.unique()), axis=0))

In [8]:
q_test = pd.DataFrame(q_test[0].unique())

In [9]:
len(q_test) + len(q_train)

4953464

In [10]:
len(q_train)

589632

In [20]:
q_test['qid1'] = range(537934,537934 + len(q_test))
q_test.columns = ['question1', 'qid1']
X_test = pd.merge(X_test, q_test, on="question1", how="left")

q_test.columns = ['question2', 'qid2']
X_test = pd.merge(X_test, q_test, on="question2", how="left")

In [11]:
X_test

,id,question1,question2,qid1,qid2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,538001,538002
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,538003,538004
2,2,What but is the best way to send money from Ch...,What you send money to China?,538005,538006
3,3,Which food not emulsifiers?,What foods fibre?,538007,538008
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,538009,538010
5,5,How are the two wheeler insurance from Bharti ...,I admire I am considering of buying insurance ...,538011,538012
6,6,How can I reduce my belly fat through a diet?,How can I reduce my lower belly fat in one month?,74513,538013
7,7,"By scrapping the 500 and 1000 rupee notes, how...",How will the recent move to declare 500 and 10...,538014,538015
8,8,What are the how best books of all time?,What are some of the military history books of...,538016,538017
9,9,After 12th years old boy and I had sex with a ...,Can a 14 old guy date a 12 year old girl?,538018,538019


In [18]:
#tmp = np.unique(np.concatenate((X_train.qid1.unique(), X_train.qid2.unique(), X_test.qid1.unique(), X_test.qid2.unique()), axis=0))
tmp = np.unique(np.concatenate((X_train.qid1.unique(), X_train.qid2.unique(), X_test.qid1.unique(), X_test.qid2.unique()), axis=0))
tmp.shape

(4789604,)

In [21]:
5000*32 / 3600.0

44.44444444444444

In [25]:
arr_train = X_train[['qid1', 'qid2']].values

In [26]:
arr_test = X_test[['qid1', 'qid2']].values

In [21]:
arr_test

array([[ 538001,  538002],
       [ 538003,  538004],
       [ 538005,  538006],
       ..., 
       [4789666, 4789667],
       [4789668, 4789669],
       [4789670, 4789671]])

In [ ]:
4789604

In [27]:
arr = []
for i in range(4789672):
    arr += [[]]
for i in arr_train:
    arr[i[0]] += [i[1]]
    arr[i[1]] += [i[0]]
for i in arr_test:
    arr[i[0]] += [i[1]]
    arr[i[1]] += [i[0]]

In [28]:
arr

[[],
 [2],
 [1, 3525867],
 [4, 282170, 380197, 488853, 1368829, 1547800, 2067446, 4044705],
 [3, 32881, 2600374],
 [6, 3714586],
 [5],
 [8],
 [7],
 [10, 109465, 480204],
 [9],
 [12],
 [11],
 [14],
 [13],
 [16],
 [15],
 [18, 38436],
 [17, 38437, 38436],
 [20],
 [19],
 [22],
 [21],
 [24],
 [23, 4697390],
 [26, 114035],
 [25, 114035],
 [28, 50277],
 [27, 50277, 2209056],
 [30, 47048, 44255, 160286, 179664, 407455, 44256, 3928258],
 [29, 179664, 261411, 47048, 44256, 2161400],
 [32, 6937, 88834, 1101, 11435, 93145, 249733, 44686, 12544, 812001, 1447047],
 [31,
  12544,
  16474,
  93145,
  88834,
  2067,
  37617,
  2066,
  11434,
  6938,
  221900,
  1101,
  46127,
  132922,
  6937,
  11435,
  1100,
  528360,
  53535,
  150421],
 [34],
 [33],
 [36],
 [35, 309051, 389457],
 [38,
  27517,
  17811,
  19621,
  88660,
  12639,
  6551,
  18296,
  17507,
  26053,
  1358,
  33412,
  21713,
  17506,
  4951,
  1357,
  17258,
  47020,
  556914,
  553263,
  226602,
  798474,
  546842],
 [37,
  4950,
  1

In [35]:
def make(idx):
    t1 = np.zeros(4789672)
    for i in arr[idx]:
        t1[i] = 1
    return t1

In [36]:
cosine(make(4254), make(4))

1.0

In [115]:
make(3)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [33]:
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

In [56]:
%%time
tmp = 0
res_train = np.array([])

for i in arr_train[:]:
    res_train = np.append(res_train, jaccard(make(i[0]), make(i[1])))
    tmp += 1
    if tmp % 10000 == 0:
        print(tmp)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
CPU times: user 3h 43min 4s, sys: 13min 14s, total: 3h 56min 18s
Wall time: 4h 20min 35s


In [55]:
2000*26.9 / 3600

14.944444444444445

In [ ]:
res_train=pd.DataFrame(res_train)
res_train.columns = ['jaccard']
res_train.to_csv("data/jaccard_train.csv", index=None)

In [ ]:
%%time
tmp = 0
res_train = np.array([])

for i in arr_test[:]:
    res_train = np.append(res_train, jaccard(make(i[0]), make(i[1])))
    tmp += 1
    if tmp % 10000 == 0:
        print(tmp)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [ ]:
res_train=pd.DataFrame(res_train)
res_train.columns = ['jaccard']
res_train.to_csv("data/jaccard_test.csv", index=None)

In [60]:
1

1

In [113]:
A = csr_matrix((4789672, 4789672))

In [32]:
%%time
A = csr_matrix((4789672, 4789672))
t = 0
for i in arr_train:
    A[i[0], i[1]] = 1
    A[i[1], i[0]] = 1
    if t  % 10000 == 0:
        print(t)
    t = t + 1
#v = np.zeros(4789672)
#v[2] = 1
#A.dot(v)

/home/fao3864/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


0


KeyboardInterrupt: 